In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
pip install transformers==3.0.0

     |████████████████████████████████| 754 kB 9.6 MB/s 
     |████████████████████████████████| 3.0 MB 51.9 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 1.2 MB 26.6 MB/s 


In [3]:
pip install pytorch-crf==0.7.2

In [4]:
#@title fix spans { form-width: "1px" }
import itertools
import string 

SPECIAL_CHARACTERS = string.whitespace

def _contiguous_ranges(span_list):
    """Extracts continguous runs [1, 2, 3, 5, 6, 7] -> [(1,3), (5,7)]."""
    output = []
    for _, span in itertools.groupby(
        enumerate(span_list), lambda p: p[1] - p[0]):
        span = list(span)
        output.append((span[0][1], span[-1][1]))
    return output

def fix_spans(spans, text, special_characters=SPECIAL_CHARACTERS):
    """Applies minor edits to trim spans and remove singletons."""
    cleaned = []
    for begin, end in _contiguous_ranges(spans):
        while text[begin] in special_characters and begin < end:
            begin += 1
        while text[end] in special_characters and begin < end:
            end -= 1
        if end - begin > 1:
            cleaned.extend(range(begin, end + 1))
    return cleaned

In [5]:
#@title spans to entities { form-width: "1px" }
import csv
import ast
import spacy

def spans_to_ents(doc, spans, label):
  """Converts span indicies into spacy entity labels."""
  started = False
  left, right, ents = 0, 0, []
  for x in doc:
    if x.pos_ == 'SPACE':
      continue
    if spans.intersection(set(range(x.idx, x.idx + len(x.text)))):
      if not started:
        left, started = x.idx, True
      right = x.idx + len(x.text)
    elif started:
      ents.append((left, right, label))
      started = False
  if started:
    ents.append((left, right, label))
  return ents

def read_datafile(filename):
  """Reads csv file with python span list and text."""
  data = []
  with open(filename) as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
      fixed = fix_spans(
          ast.literal_eval(row['spans']), row['text'])
      data.append((fixed, row['text']))
  return data

In [6]:
#@title build NER structure { form-width: "1px" }
import nltk

nltk.download('punkt')

def build_ner_data_structure(data):
  ner_labels = []
  ner_texts = []
  skip_index = []
  for i in range(len(data)):
    skip_sample = False
    sent_o = data[i][2]
    sent = data[i][2]
    words_tmp = nltk.word_tokenize(sent)
    words = []
    for j in words_tmp:
      if sent.find(j) > -1:
        words.append(j)
    word_level_labels = []
    bias = 0
    for word in words:
      index = sent.find(word)
      if index == -1:
        # print("error")
        skip_sample = True
        continue
      flag = True
      for entity in data[i][1]["entities"]:
        if index + bias < entity[1] and index + bias >= entity[0]:
          word_level_labels.append("I")
          flag = False
          break
      if flag:
        word_level_labels.append("O")
      bias = bias + index + len(word)
      sent = sent[index + len(word):]
    if skip_sample:
      print("ship this sample")
      skip_index.append(i)
      continue
    ner_texts.append(words)
    ner_labels.append(word_level_labels)
  
  if len(ner_labels) != len(ner_texts):
    print("num of samples inconsist!")
    quit()
  for i in range(len(ner_labels)):
    if len(ner_labels[i]) != len(ner_texts[i]):
      print("sentence len inconsist!")
      quit()

  return ner_texts, ner_labels, skip_index

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
#@title build dataset { form-width: "1px" }
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import BertModel, BertTokenizerFast
import numpy as np

label2id = {
    "O": 0,
    "I": 1
}
id2label = {_id: _label for _label, _id in list(label2id.items())}

class NERDataset(Dataset):
  def __init__(self, sentences, labels, max_len, origional_data):
    self.tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", do_lower_case=True)
    self.label2id = label2id
    self.id2label = {_id: _label for _label, _id in list(label2id.items())}
    self.sentences = sentences
    self.labels = labels
    self.origional_data = origional_data
    self.max_seq_len = max_len
    self.dataset = self.preprocess(sentences, labels, origional_data)
    self.word_pad_idx = 0
    self.label_pad_idx = -100
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

  def preprocess(self, sentences, labels, origional_data):
    data = []
    for sentence, label, origional_data_item in zip(sentences, labels, origional_data):
        data.append((sentence, label, origional_data_item))
    return data

  def __getitem__(self, idx):
    sentence = self.dataset[idx][0]
    word_labels = self.dataset[idx][1]
    origional_sentence = self.dataset[idx][2][1]
    origional_label = self.dataset[idx][2][0]
    encoding = self.tokenizer(sentence,
                  is_pretokenized=True, 
                  return_offsets_mapping=True, 
                  padding='max_length', 
                  truncation=True, 
                  max_length=self.max_seq_len)
    labels = [self.label2id[label] for label in word_labels] 
    length = 0
    for i in encoding["input_ids"]:
      if i != 102:
        length = length + 1
      else:
        break
    encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
    i = 0
    for idx, mapping in enumerate(encoding["offset_mapping"]):
      if mapping[0] == 0 and mapping[1] != 0:
        encoded_labels[idx] = labels[i]
        i += 1
      elif mapping[0] > 0:
        encoded_labels[idx] = labels[i-1]
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
    item['labels'] = torch.as_tensor(encoded_labels)
    item['sentences'] = sentence
    item['origional_sentences'] = origional_sentence
    item['origional_labels'] = origional_label
    item["tokens_len"] = length
    return item

  def __len__(self):
    return len(self.dataset)
  
  def collate_fn(self, batch):
    sentences = [x["input_ids"] for x in batch]
    labels = [x["labels"] for x in batch]
    sentence = [x["sentences"] for x in batch]
    origional_sentence = [x["origional_sentences"] for x in batch]
    origional_label = [x["origional_labels"] for x in batch]
    length = [x["tokens_len"] for x in batch]
    offset = [x["offset_mapping"] for x in batch]
    sentences = [x["input_ids"] for x in batch]
    attention_masks = [x["attention_mask"] for x in batch]
    # token_type_ids = [x["token_type_ids"] for x in batch]

    batch_len = len(sentences)

    batch_data = self.word_pad_idx * np.ones((batch_len, self.max_seq_len))
    batch_labels = self.label_pad_idx * np.ones((batch_len, self.max_seq_len))
    batch_attention_masks = self.word_pad_idx * np.ones((batch_len, self.max_seq_len))
    for i in range(batch_len):
      cur_len = len(sentences[i])
      batch_data[i][:cur_len] = sentences[i]
      cur_tags_len = len(labels[i])
      batch_labels[i][:cur_tags_len] = labels[i]
      cur_tags_len = len(attention_masks[i])
      batch_attention_masks[i][:cur_tags_len] = attention_masks[i]

    # convert data to torch LongTensors
    batch_data = torch.tensor(batch_data, dtype=torch.long)
    batch_attention_masks = torch.tensor([item.detach().numpy() for item in attention_masks], dtype=torch.long)
    batch_labels = torch.tensor(batch_labels, dtype=torch.long)
    # batch_token_type_ids = torch.tensor([item.detach().numpy() for item in token_type_ids], dtype=torch.long)

    # shift tensors to GPU if available
    # batch_data = batch_data.to(self.device)
    # batch_attention_masks = batch_attention_masks.to(self.device)
    # batch_label_starts = batch_label_starts.to(self.device)
    # batch_labels = batch_labels.to(self.device)

    return [batch_data, batch_attention_masks, batch_labels, sentence, origional_sentence, origional_label, length, offset]

In [8]:
#@title crf layer
from typing import List,Optional
 
class CRF(nn.Module):
    """
    Attributes:
        start_transitions (`~torch.nn.Parameter`): Start transition score tensor of size
            ``(num_tags,)``.
        end_transitions (`~torch.nn.Parameter`): End transition score tensor of size
            ``(num_tags,)``.
        transitions (`~torch.nn.Parameter`): Transition score tensor of size
            ``(num_tags, num_tags)``.
    .. [LMP01] Lafferty, J., McCallum, A., Pereira, F. (2001).
       "Conditional random fields: Probabilistic models for segmenting and
       labeling sequence data". *Proc. 18th International Conf. on Machine
       Learning*. Morgan Kaufmann. pp. 282–289.
    .. _Viterbi algorithm: https://en.wikipedia.org/wiki/Viterbi_algorithm
    """
    def __init__(self, num_tags: int, batch_first: bool = False) -> None:
        if num_tags <= 0:
            raise ValueError(f'invalid number of tags: {num_tags}')
        super().__init__()
        self.num_tags = num_tags
        self.batch_first = batch_first
        self.start_transitions = nn.Parameter(torch.empty(num_tags))
        self.end_transitions = nn.Parameter(torch.empty(num_tags))
        self.transitions = nn.Parameter(torch.empty(num_tags, num_tags))
 
        self.reset_parameters()
 
    def reset_parameters(self):
        """
        Initialize the transition parameters.
        The parameters will be initialized randomly from a uniform distribution
        between -0.1 and 0.1.
        :return:
        """
        nn.init.uniform_(self.start_transitions,-0.1,0.1)
        nn.init.uniform_(self.end_transitions,-0.1,0.1)
        nn.init.uniform_(self.transitions,-0.1,0.1)
 
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(num_tags={self.num_tags})'
 
 
    def forward(self, emissions: torch.Tensor,
                tags: torch.LongTensor,
                mask: Optional[torch.ByteTensor] = None,
                reduction: str = 'mean'):
        """Compute the conditional log likelihood of a sequence of tags given emission scores.
                Args:
                    emissions (`~torch.Tensor`): Emission score tensor of size
                        ``(seq_length, batch_size, num_tags)`` if ``batch_first`` is ``False``,
                        ``(batch_size, seq_length, num_tags)`` otherwise.
                    tags (`~torch.LongTensor`): Sequence of tags tensor of size
                        ``(seq_length, batch_size)`` if ``batch_first`` is ``False``,
                        ``(batch_size, seq_length)`` otherwise.
                    mask (`~torch.ByteTensor`): Mask tensor of size ``(seq_length, batch_size)``
                        if ``batch_first`` is ``False``, ``(batch_size, seq_length)`` otherwise.
                    reduction: Specifies  the reduction to apply to the output:
                        ``none|sum|mean|token_mean``. ``none``: no reduction will be applied.
                        ``sum``: the output will be summed over batches. ``mean``: the output will be
                        averaged over batches. ``token_mean``: the output will be averaged over tokens.
                Returns:
                    `~torch.Tensor`: The log likelihood. This will have size ``(batch_size,)`` if
                    reduction is ``none``, ``()`` otherwise.
                """
        if reduction not in ('none', 'sum', 'mean', 'token_mean'):
            raise ValueError(f'invalid reduction: {reduction}')
        if mask is None:
            mask = torch.ones_like(tags, dtype=torch.uint8, device=tags.device)
        if mask.dtype != torch.uint8:
            mask = mask.byte()
        self._validate(emissions, tags=tags, mask=mask)\
 
        if self.batch_first:
            emissions = emissions.transpose(0, 1)
            tags = tags.transpose(0, 1)
            mask = mask.transpose(0, 1)
            # shape: (batch_size,)
            numerator = self._compute_score(emissions, tags, mask)
            # shape: (batch_size,)
            denominator = self._compute_normalizer(emissions, mask)
            # shape: (batch_size,)
            llh = numerator - denominator
 
            if reduction == 'none':
                return llh
            if reduction == 'sum':
                return llh.sum()
            if reduction == 'mean':
                return llh.mean()
            return llh.sum() / mask.float().sum()
 
 
    def decode(self, emissions: torch.Tensor,
               mask: Optional[torch.ByteTensor] = None,
               nbest: Optional[int] = None,
               pad_tag: Optional[int] = None) -> List[List[List[int]]]:
        """Find the most likely tag sequence using Viterbi algorithm.
        Args:
            emissions (`~torch.Tensor`): Emission score tensor of size
                ``(seq_length, batch_size, num_tags)`` if ``batch_first`` is ``False``,
                ``(batch_size, seq_length, num_tags)`` otherwise.
            mask (`~torch.ByteTensor`): Mask tensor of size ``(seq_length, batch_size)``
                if ``batch_first`` is ``False``, ``(batch_size, seq_length)`` otherwise.
            nbest (`int`): Number of most probable paths for each sequence
            pad_tag (`int`): Tag at padded positions. Often input varies in length and
                the length will be padded to the maximum length in the batch. Tags at
                the padded positions will be assigned with a padding tag, i.e. `pad_tag`
        Returns:
            A PyTorch tensor of the best tag sequence for each batch of shape
            (nbest, batch_size, seq_length)
        """
        if nbest is None:
            nbest = 1
        if mask is None:
            mask = torch.ones(emissions.shape[:2], dtype=torch.uint8,
                              device=emissions.device)
        if mask.dtype != torch.uint8:
            mask = mask.byte()
        self._validate(emissions, mask=mask)
 
        if self.batch_first:
            emissions = emissions.transpose(0, 1)
            mask = mask.transpose(0, 1)
 
        if nbest == 1:
            return self._viterbi_decode(emissions, mask, pad_tag).unsqueeze(0)
        return self._viterbi_decode_nbest(emissions, mask, nbest, pad_tag)
 
 
    def _validate(self, emissions: torch.Tensor,
                  tags: Optional[torch.LongTensor] = None,
                  mask: Optional[torch.ByteTensor] = None) -> None:
        if emissions.dim() != 3:
            raise ValueError(f'emissions must have dimension of 3, got {emissions.dim()}')
        if emissions.size(2) != self.num_tags:
            raise ValueError(
                f'expected last dimension of emissions is {self.num_tags}, '
                f'got {emissions.size(2)}')
 
        if tags is not None:
            if emissions.shape[:2] != tags.shape:
                raise ValueError(
                    'the first two dimensions of emissions and tags must match, '
                    f'got {tuple(emissions.shape[:2])} and {tuple(tags.shape)}')
 
        if mask is not None:
            if emissions.shape[:2] != mask.shape:
                raise ValueError(
                    'the first two dimensions of emissions and mask must match, '
                    f'got {tuple(emissions.shape[:2])} and {tuple(mask.shape)}')
            no_empty_seq = not self.batch_first and mask[0].all()
            no_empty_seq_bf = self.batch_first and mask[:, 0].all()
            if not no_empty_seq and not no_empty_seq_bf:
                raise ValueError('mask of the first timestep must all be on')
 
    def _compute_score(self, emissions: torch.Tensor,
                       tags: torch.LongTensor,
                       mask: torch.ByteTensor) -> torch.Tensor:
        # emissions: (seq_length, batch_size, num_tags)
        # tags: (seq_length, batch_size)
        # mask: (seq_length, batch_size)
        seq_length, batch_size = tags.shape
        mask = mask.float()
 
        # Start transition score and first emission
        # shape: (batch_size,)
        score = self.start_transitions[tags[0]]
        score += emissions[0, torch.arange(batch_size), tags[0]]
 
        for i in range(1, seq_length):
            # Transition score to next tag, only added if next timestep is valid (mask == 1)
            # shape: (batch_size,)
            score += self.transitions[tags[i - 1], tags[i]] * mask[i]
 
            # Emission score for next tag, only added if next timestep is valid (mask == 1)
            # shape: (batch_size,)
            score += emissions[i, torch.arange(batch_size), tags[i]] * mask[i]
 
        # End transition score
        # shape: (batch_size,)
        seq_ends = mask.long().sum(dim=0) - 1
        # shape: (batch_size,)
        last_tags = tags[seq_ends, torch.arange(batch_size)]
        # shape: (batch_size,)
        score += self.end_transitions[last_tags]
 
        return score
 
 
    def _compute_normalizer(self, emissions: torch.Tensor,
                            mask: torch.ByteTensor) -> torch.Tensor:
        # emissions: (seq_length, batch_size, num_tags)
        # mask: (seq_length, batch_size)
        seq_length = emissions.size(0)
 
        # Start transition score and first emission; score has size of
        # (batch_size, num_tags) where for each batch, the j-th column stores
        # the score that the first timestep has tag j
        # shape: (batch_size, num_tags)
        score = self.start_transitions + emissions[0]
 
        for i in range(1, seq_length):
            # Broadcast score for every possible next tag
            # shape: (batch_size, num_tags, 1)
            broadcast_score = score.unsqueeze(2)
 
            # Broadcast emission score for every possible current tag
            # shape: (batch_size, 1, num_tags)
            broadcast_emissions = emissions[i].unsqueeze(1)
 
            # Compute the score tensor of size (batch_size, num_tags, num_tags) where
            # for each sample, entry at row i and column j stores the sum of scores of all
            # possible tag sequences so far that end with transitioning from tag i to tag j
            # and emitting
            # shape: (batch_size, num_tags, num_tags)
            next_score = broadcast_score + self.transitions + broadcast_emissions
 
            # Sum over all possible current tags, but we're in score space, so a sum
            # becomes a log-sum-exp: for each sample, entry i stores the sum of scores of
            # all possible tag sequences so far, that end in tag i
            # shape: (batch_size, num_tags)
            next_score = torch.logsumexp(next_score, dim=1)
 
            # Set score to the next score if this timestep is valid (mask == 1)
            # shape: (batch_size, num_tags)
            score = torch.where(mask[i].unsqueeze(1), next_score, score)
 
        # End transition score
        # shape: (batch_size, num_tags)
        score += self.end_transitions
 
        # Sum (log-sum-exp) over all possible tags
        # shape: (batch_size,)
        return torch.logsumexp(score, dim=1)
 
    def _viterbi_decode(self, emissions: torch.FloatTensor,
                        mask: torch.ByteTensor,
                        pad_tag: Optional[int] = None) -> List[List[int]]:
        # emissions: (seq_length, batch_size, num_tags)
        # mask: (seq_length, batch_size)
        # return: (batch_size, seq_length)
        if pad_tag is None:
            pad_tag = 0
 
        device = emissions.device
        seq_length, batch_size = mask.shape
 
        # Start transition and first emission
        # shape: (batch_size, num_tags)
        score = self.start_transitions + emissions[0]
        history_idx = torch.zeros((seq_length, batch_size, self.num_tags),
                                  dtype=torch.long, device=device)
        oor_idx = torch.zeros((batch_size, self.num_tags),
                              dtype=torch.long, device=device)
        oor_tag = torch.full((seq_length, batch_size), pad_tag,
                             dtype=torch.long, device=device)
 
        # - score is a tensor of size (batch_size, num_tags) where for every batch,
        #   value at column j stores the score of the best tag sequence so far that ends
        #   with tag j
        # - history_idx saves where the best tags candidate transitioned from; this is used
        #   when we trace back the best tag sequence
        # - oor_idx saves the best tags candidate transitioned from at the positions
        #   where mask is 0, i.e. out of range (oor)
 
        # Viterbi algorithm recursive case: we compute the score of the best tag sequence
        # for every possible next tag
        for i in range(1, seq_length):
            # Broadcast viterbi score for every possible next tag
            # shape: (batch_size, num_tags, 1)
            broadcast_score = score.unsqueeze(2)
 
            # Broadcast emission score for every possible current tag
            # shape: (batch_size, 1, num_tags)
            broadcast_emission = emissions[i].unsqueeze(1)
 
            # Compute the score tensor of size (batch_size, num_tags, num_tags) where
            # for each sample, entry at row i and column j stores the score of the best
            # tag sequence so far that ends with transitioning from tag i to tag j and emitting
            # shape: (batch_size, num_tags, num_tags)
            next_score = broadcast_score + self.transitions + broadcast_emission
 
            # Find the maximum score over all possible current tag
            # shape: (batch_size, num_tags)
            next_score, indices = next_score.max(dim=1)
 
            # Set score to the next score if this timestep is valid (mask == 1)
            # and save the index that produces the next score
            # shape: (batch_size, num_tags)
            score = torch.where(mask[i].unsqueeze(-1), next_score, score)
            indices = torch.where(mask[i].unsqueeze(-1), indices, oor_idx)
            history_idx[i - 1] = indices
 
        # End transition score
        # shape: (batch_size, num_tags)
        end_score = score + self.end_transitions
        _, end_tag = end_score.max(dim=1)
 
        # shape: (batch_size,)
        seq_ends = mask.long().sum(dim=0) - 1
 
        # insert the best tag at each sequence end (last position with mask == 1)
        history_idx = history_idx.transpose(1, 0).contiguous()
        history_idx.scatter_(1, seq_ends.view(-1, 1, 1).expand(-1, 1, self.num_tags),
                             end_tag.view(-1, 1, 1).expand(-1, 1, self.num_tags))
        history_idx = history_idx.transpose(1, 0).contiguous()
 
        # The most probable path for each sequence
        best_tags_arr = torch.zeros((seq_length, batch_size),
                                    dtype=torch.long, device=device)
        best_tags = torch.zeros(batch_size, 1, dtype=torch.long, device=device)
        for idx in range(seq_length - 1, -1, -1):
            best_tags = torch.gather(history_idx[idx], 1, best_tags)
            best_tags_arr[idx] = best_tags.data.view(batch_size)
 
        return torch.where(mask, best_tags_arr, oor_tag).transpose(0, 1)
 
    def _viterbi_decode_nbest(self, emissions: torch.FloatTensor,
                              mask: torch.ByteTensor,
                              nbest: int,
                              pad_tag: Optional[int] = None) -> List[List[List[int]]]:
        # emissions: (seq_length, batch_size, num_tags)
        # mask: (seq_length, batch_size)
        # return: (nbest, batch_size, seq_length)
        if pad_tag is None:
            pad_tag = 0
 
        device = emissions.device
        seq_length, batch_size = mask.shape
 
        # Start transition and first emission
        # shape: (batch_size, num_tags)
        score = self.start_transitions + emissions[0]
        history_idx = torch.zeros((seq_length, batch_size, self.num_tags, nbest),
                                  dtype=torch.long, device=device)
        oor_idx = torch.zeros((batch_size, self.num_tags, nbest),
                              dtype=torch.long, device=device)
        oor_tag = torch.full((seq_length, batch_size, nbest), pad_tag,
                             dtype=torch.long, device=device)
 
        # + score is a tensor of size (batch_size, num_tags) where for every batch,
        #   value at column j stores the score of the best tag sequence so far that ends
        #   with tag j
        # + history_idx saves where the best tags candidate transitioned from; this is used
        #   when we trace back the best tag sequence
        # - oor_idx saves the best tags candidate transitioned from at the positions
        #   where mask is 0, i.e. out of range (oor)
 
        # Viterbi algorithm recursive case: we compute the score of the best tag sequence
        # for every possible next tag
        for i in range(1, seq_length):
            if i == 1:
                broadcast_score = score.unsqueeze(-1)
                broadcast_emission = emissions[i].unsqueeze(1)
                # shape: (batch_size, num_tags, num_tags)
                next_score = broadcast_score + self.transitions + broadcast_emission
            else:
                broadcast_score = score.unsqueeze(-1)
                broadcast_emission = emissions[i].unsqueeze(1).unsqueeze(2)
                # shape: (batch_size, num_tags, nbest, num_tags)
                next_score = broadcast_score + self.transitions.unsqueeze(1) + broadcast_emission
 
            # Find the top `nbest` maximum score over all possible current tag
            # shape: (batch_size, nbest, num_tags)
            next_score, indices = next_score.view(batch_size, -1, self.num_tags).topk(nbest, dim=1)
 
            if i == 1:
                score = score.unsqueeze(-1).expand(-1, -1, nbest)
                indices = indices * nbest
 
            # convert to shape: (batch_size, num_tags, nbest)
            next_score = next_score.transpose(2, 1)
            indices = indices.transpose(2, 1)
 
            # Set score to the next score if this timestep is valid (mask == 1)
            # and save the index that produces the next score
            # shape: (batch_size, num_tags, nbest)
            score = torch.where(mask[i].unsqueeze(-1).unsqueeze(-1), next_score, score)
            indices = torch.where(mask[i].unsqueeze(-1).unsqueeze(-1), indices, oor_idx)
            history_idx[i - 1] = indices
 
        # End transition score shape: (batch_size, num_tags, nbest)
        end_score = score + self.end_transitions.unsqueeze(-1)
        _, end_tag = end_score.view(batch_size, -1).topk(nbest, dim=1)
 
        # shape: (batch_size,)
        seq_ends = mask.long().sum(dim=0) - 1
 
        # insert the best tag at each sequence end (last position with mask == 1)
        history_idx = history_idx.transpose(1, 0).contiguous()
        history_idx.scatter_(1, seq_ends.view(-1, 1, 1, 1).expand(-1, 1, self.num_tags, nbest),
                             end_tag.view(-1, 1, 1, nbest).expand(-1, 1, self.num_tags, nbest))
        history_idx = history_idx.transpose(1, 0).contiguous()
 
        # The most probable path for each sequence
        best_tags_arr = torch.zeros((seq_length, batch_size, nbest),
                                    dtype=torch.long, device=device)
        best_tags = torch.arange(nbest, dtype=torch.long, device=device) \
                         .view(1, -1).expand(batch_size, -1)
        for idx in range(seq_length - 1, -1, -1):
            best_tags = torch.gather(history_idx[idx].view(batch_size, -1), 1, best_tags)
            best_tags_arr[idx] = best_tags.data.view(batch_size, -1) // nbest
 
        return torch.where(mask.unsqueeze(-1), best_tags_arr, oor_tag).permute(2, 1, 0)

In [9]:
#@title model { form-width: "1px" }
from transformers.modeling_bert import *
from torch.nn.utils.rnn import pad_sequence

class BertNER(BertPreTrainedModel):
  def __init__(self,config):
    super(BertNER, self).__init__(config)
    self.num_labels = 2
    self.bert = BertModel(config)
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768, self.num_labels)
    self.crf = CRF(self.num_labels, batch_first=True)
    self.init_weights()

  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, batch_len = None):
    outputs = self.bert(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
    sequence_output = outputs[0]
    origin_sequence_output = [layer[1:]
                              for layer, end in zip(sequence_output, batch_len)]
    padded_sequence_output = pad_sequence(origin_sequence_output, batch_first=True)
    dropout_output = self.dropout(padded_sequence_output)
    logits = self.classifier(dropout_output)
    outputs = (logits,)

    if labels is not None:
      origional_labels = [layer[1:]
                          for layer, end in zip(labels, batch_len)]
      for i in range(len(origional_labels)):
        zero = torch.zeros_like(origional_labels[i])
        origional_labels[i] = torch.where(origional_labels[i]>-1, origional_labels[i], zero)
      padded_labels = pad_sequence(origional_labels, batch_first=True)
      loss_mask = padded_labels.gt(-100)
      loss = self.crf(logits, padded_labels, mask=loss_mask) * (-1)
      outputs = (loss,) + outputs

    return outputs

class BertLstmNER(BertPreTrainedModel):
  def __init__(self,config):
    super(BertLstmNER, self).__init__(config)
    self.num_labels = 2
    self.bert = BertModel(config)
    self.dropout = nn.Dropout(0.5)
    self.lstm = nn.LSTM(768, 384, batch_first=True, bidirectional=True, num_layers = 2)
    self.classifier = nn.Linear(768, self.num_labels)
    self.crf = CRF(self.num_labels, batch_first=True)
    self.init_weights()

  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, batch_len = None):
    outputs = self.bert(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
    sequence_output = outputs[0]
    origin_sequence_output = [layer[1:]
                              for layer, end in zip(sequence_output, batch_len)]
    padded_sequence_output = pad_sequence(origin_sequence_output, batch_first=True)
    lstm_out, hidden = self.lstm(padded_sequence_output)
    dropout_output = self.dropout(lstm_out)
    logits = self.classifier(dropout_output)
    outputs = (logits,)

    if labels is not None:
      origional_labels = [layer[1:]
                          for layer, end in zip(labels, batch_len)]
      for i in range(len(origional_labels)):
        zero = torch.zeros_like(origional_labels[i])
        origional_labels[i] = torch.where(origional_labels[i]>-1, origional_labels[i], zero)
      padded_labels = pad_sequence(origional_labels, batch_first=True)
      loss_mask = padded_labels.gt(-100)
      loss = self.crf(logits, padded_labels, mask=loss_mask) * (-1)
      outputs = (loss,) + outputs

    return outputs

class BertAvgNER(BertPreTrainedModel):
  def __init__(self,config):
    super(BertAvgNER, self).__init__(config)
    self.num_labels = 2
    self.bert = BertModel(config)
    self.dropout = nn.Dropout(0.5)
    self.lstm = nn.LSTM(2304, 1152, batch_first=True, bidirectional=True, num_layers = 2)
    self.classifier = nn.Linear(2304, self.num_labels)
    self.crf = CRF(self.num_labels, batch_first=True)
    self.init_weights()

  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, batch_len = None):
    outputs = self.bert(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        output_hidden_states=True)
    
    hidden_states = outputs[2]
    sequence_output = torch.cat((hidden_states[3], hidden_states[7], hidden_states[11]), 2)
    origin_sequence_output = [layer[1:]
                              for layer, end in zip(sequence_output, batch_len)]
    padded_sequence_output = pad_sequence(origin_sequence_output, batch_first=True)
    lstm_out, hidden = self.lstm(padded_sequence_output)
    dropout_output = self.dropout(lstm_out)
    logits = self.classifier(dropout_output)
    outputs = (logits,)

    if labels is not None:
      origional_labels = [layer[1:]
                          for layer, end in zip(labels, batch_len)]
      for i in range(len(origional_labels)):
        zero = torch.zeros_like(origional_labels[i])
        origional_labels[i] = torch.where(origional_labels[i]>-1, origional_labels[i], zero)
      padded_labels = pad_sequence(origional_labels, batch_first=True)
      loss_mask = padded_labels.gt(-100)
      loss = self.crf(logits, padded_labels, mask=loss_mask) * (-1)
      outputs = (loss,) + outputs

    return outputs

In [10]:
#@title f1 { form-width: "1px" }
import sys
import os
import os.path
from scipy.stats import sem
import numpy as np
from ast import literal_eval

def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)


# def evaluate(pred, gold):
#     """
#     Based on https://github.com/felipebravom/EmoInt/blob/master/codalab/scoring_program/evaluation.py
#     :param pred: file with predictions
#     :param gold: file with ground truth
#     :return:
#     """

#     # lists storing gold and prediction scores
#     scores = []
#     for i in range(len(pred)):
#         scores.append(f1(pred[i], gold[i]))
#         print(pred[i])
#         print(gold[i])
#         print(f1(pred[i], gold[i]))
            
#     return (np.mean(scores), sem(scores))

In [11]:
#@title convert word label to character level { form-width: "1px" }
def get_char_level_label(data):
  labels = []
  length = data["length"]
  tags = data["tags"][1:length]
  offset = data["offset"]
  words = data["words"]
  sentence = data["sentence"]
  index = 0
  bias = 0
  for i in range(length - 1):
    if offset[i+1][0] == 0:
      subword = words[index][offset[i+1][0]:offset[i+1][1]]
    elif offset[i+1][0] > 0:
      subword = words[index][offset[i+1][0]:offset[i+1][1]]
    if offset[i+1][1] == len(words[index]):
      index = index + 1
    elif offset[i+1][1] > len(words[index]):
      print("error")
    bias = sentence.find(subword) + bias

    if tags[i] == "I":
      pos = bias
      for i in range(len(subword)):
        labels.append(pos)
        pos = pos + 1
    bias = bias + len(subword)
    sentence = sentence[sentence.find(subword)+len(subword):]

  for i in range(1, len(sentence)-1):
    if sentence[i] == " ":
      if (i-1 in labels) and (i+1 in labels):
        labels.append(i)
    
  
  return labels

In [12]:
#@title train dev test func { form-width: "1px" }

import logging
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train_epoch(train_loader, model, optimizer, scheduler, epoch):
  model.train()
  train_losses = 0
  pred_tags = []
  gold_tags = []
  pred_labels = []
  true_labels = []
  for idx, batch_samples in enumerate(tqdm(train_loader)):
    batch_data, batch_masks, batch_labels, sentence, origional_sentences, origional_labels, length, offset = batch_samples
    # shift tensors to GPU if available
    batch_data = batch_data.to(device)
    batch_masks = batch_masks.to(device)
    batch_labels = batch_labels.to(device)

    loss = model(batch_data,
                token_type_ids=None, 
                attention_mask=batch_masks, 
                labels=batch_labels,
                batch_len = length)
    loss = loss[0]
    train_losses += loss.item()
    model.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=5)
    optimizer.step()
    scheduler.step()

  train_loss = float(train_losses) / len(train_loader)
  print("Epoch: {}, train loss: {}".format(epoch, train_loss))

def dev_epoch(dev_loader, model, optimizer, scheduler, epoch):
  model.eval()
  dev_losses = 0
  pred_tags = []
  gold_tags = []
  pred_labels = []
  true_labels = []
  for idx, batch_samples in enumerate(tqdm(dev_loader)):
    batch_data, batch_masks, batch_labels, sentence, origional_sentences, origional_labels, length, offset = batch_samples
    # shift tensors to GPU if available
    batch_data = batch_data.to(device)
    batch_masks = batch_masks.to(device)
    batch_labels = batch_labels.to(device)

    batch_output = model(batch_data,
                        token_type_ids=None, 
                        attention_mask=batch_masks,
                        batch_len = length)
    batch_output= batch_output[0].detach().cpu().numpy()
    batch_labels = batch_labels.to('cpu').numpy()
    pred_tags = [[id2label.get(idx.item()) for idx in indices] for indices in np.argmax(batch_output, axis=2)]
    # gold_tags.extend([[id2label.get(idx.item()) if idx.item() >= 0 else "O" for idx in indices] for indices in batch_labels])

    for i in range(len(length)):
      data_tobe_convert = {}
      data_tobe_convert["length"] = length[i]
      data_tobe_convert["tags"] = ["O"] + pred_tags[i]
      data_tobe_convert["offset"] = offset[i]
      data_tobe_convert["sentence"] = origional_sentences[i]
      data_tobe_convert["words"] = sentence[i]
      pred_labels.append(get_char_level_label(data_tobe_convert))
      true_labels.append(origional_labels[i])

  scores = [] 
  for i in range(len(pred_labels)):
    scores.append(f1(pred_labels[i],true_labels[i]))

  scores = np.mean(scores)

  print("Epoch: {}, f1: {}".format(epoch, scores * 100))

def train_model(train_loader, dev_loader, model, optimizer, scheduler, epoch_num):
  for epoch in range(1, epoch_num + 1):
    train_epoch(train_loader, model, optimizer, scheduler, epoch)
    dev_epoch(dev_loader, model, optimizer, scheduler, epoch)
  print("Training Finished!")

def test_model(test_loader, model, optimizer, scheduler):
  model.eval()
  pred_tags = []
  gold_tags = []
  pred_labels = []
  true_labels = []
  for idx, batch_samples in enumerate(tqdm(test_loader)):
    batch_data, batch_masks, batch_labels, sentence, origional_sentences, origional_labels, length, offset = batch_samples
    # shift tensors to GPU if available
    batch_data = batch_data.to(device)
    batch_masks = batch_masks.to(device)
    batch_labels = batch_labels.to(device)

    batch_output = model(batch_data,
                        token_type_ids=None, 
                        attention_mask=batch_masks,
                        batch_len = length)
    batch_output= batch_output[0].detach().cpu().numpy()
    batch_labels = batch_labels.to('cpu').numpy()
    pred_tags = [[id2label.get(idx.item()) for idx in indices] for indices in np.argmax(batch_output, axis=2)]
    # gold_tags.extend([[id2label.get(idx.item()) if idx.item() >= 0 else "O" for idx in indices] for indices in batch_labels])

    for i in range(len(length)):
      data_tobe_convert = {}
      data_tobe_convert["length"] = length[i]
      data_tobe_convert["tags"] = ["O"] + pred_tags[i]
      data_tobe_convert["offset"] = offset[i]
      data_tobe_convert["sentence"] = origional_sentences[i]
      data_tobe_convert["words"] = sentence[i]
      pred_labels.append(get_char_level_label(data_tobe_convert))
      true_labels.append(origional_labels[i])

  scores = [] 
  for i in range(len(pred_labels)):
    scores.append(f1(pred_labels[i],true_labels[i]))

  scores = np.mean(scores)

  print("test f1: {}".format(scores * 100))

In [19]:
# @title load data { form-width: "1px" }
train = read_datafile('/content/drive/My Drive/5018_ToxicSpans/datasets/tsd_train.csv')
nlp = spacy.load("en_core_web_sm")
print('preparing training data')
training_data = []
for n, (spans, text) in enumerate(train):
  doc = nlp(text)
  ents = spans_to_ents(doc, set(spans), 'TOXIC')
  training_data.append((doc.text, {'entities': ents}, text))
X, y, skiped = build_ner_data_structure(training_data)
print(skiped)
train_clean = []
for i in range(len(train)):
  if i not in skiped:
    train_clean.append(train[i])
training_set = NERDataset(X, y, 96, train_clean)

trial = read_datafile('/content/drive/My Drive/5018_ToxicSpans/datasets/tsd_trial.csv')
nlp = spacy.load("en_core_web_sm")
print('preparing trail data')
trial_data = []
for n, (spans, text) in enumerate(trial):
  doc = nlp(text)
  ents = spans_to_ents(doc, set(spans), 'TOXIC')
  trial_data.append((doc.text, {'entities': ents}, text))
X, y, skiped = build_ner_data_structure(trial_data)
trial_clean = []
for i in range(len(trial)):
  if i not in skiped:
    trial_clean.append(trial[i])
trial_set = NERDataset(X, y, 96, trial_clean)

test= read_datafile('/content/drive/My Drive/5018_ToxicSpans/datasets/tsd_test.csv')
nlp = spacy.load("en_core_web_sm")
print('preparing test data')
test_data = []
for n, (spans, text) in enumerate(test):
  doc = nlp(text)
  ents = spans_to_ents(doc, set(spans), 'TOXIC')
  test_data.append((doc.text, {'entities': ents}, text))
X, y, skiped = build_ner_data_structure(test_data)
test_clean = []
for i in range(len(test)):
  if i not in skiped:
    test_clean.append(test[i])
test_set = NERDataset(X, y, 96, test_clean)

preparing training data
ship this sample
[6819]
preparing trail data
preparing test data


In [20]:
#@title for_pseudo_label { form-width: "1px" }
import copy

nlp = spacy.load("en_core_web_sm")
print('preparing training data')
training_data = []
for n, (spans, text) in enumerate(train_clean):
  doc = nlp(text)
  ents = spans_to_ents(doc, set(spans), 'TOXIC')
  training_data.append((doc.text, {'entities': ents}, text))
X,y,skiped = build_ner_data_structure(training_data)

label_file = open('/content/drive/My Drive/5018_ToxicSpans/label_files/bert_avg.csv')
lines = label_file.readlines()
pseudo_labels = []
for line in lines:
  tags = line.split(",")[:-1]
  clean_tags = []
  for tag in tags:
    clean_tags.append(int(tag))
  pseudo_labels.append(clean_tags)
label_file.close()

test_data = []
test_pseudo = copy.deepcopy(test)
for i, (spans, text) in enumerate(test):
  test_pseudo[i] = (pseudo_labels[i], text)
  doc = nlp(text)
  ents = spans_to_ents(doc, set(pseudo_labels[i]), 'TOXIC')
  test_data.append((doc.text, {'entities': ents}, text))
X2,y2,skiped = build_ner_data_structure(test_data)

X_ext = X + X2
y_ext = y + y2
train_ext = train_clean + test_pseudo
print(len(X_ext),len(y_ext),len(train_ext))

training_set_ext = NERDataset(X_ext, y_ext, 96, train_ext)

preparing training data
9938 9938 9938


In [17]:
#@title start training { form-width: "1px" }
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers.optimization import get_cosine_schedule_with_warmup, AdamW

train_loader = DataLoader(training_set, batch_size=32, shuffle=False, collate_fn=training_set.collate_fn)
train_loader_ext = DataLoader(training_set_ext, batch_size=32, shuffle=False, collate_fn=training_set.collate_fn)
trial_loader = DataLoader(trial_set, batch_size=32, shuffle=False, collate_fn=trial_set.collate_fn)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, collate_fn=test_set.collate_fn)

model = BertAvgNER.from_pretrained("bert-base-uncased")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model.to(device)
bert_optimizer = list(model.bert.named_parameters())
classifier_optimizer = list(model.classifier.named_parameters())
lstm_optimizer = list(model.lstm.named_parameters())
crf_optimizer = list(model.crf.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in bert_optimizer if not any(nd in n for nd in no_decay)],
      'lr': 1e-5, 'weight_decay': 0.01},
    {'params': [p for n, p in bert_optimizer if any(nd in n for nd in no_decay)],
      'lr': 1e-5, 'weight_decay': 0.0},
    {'params': [p for n, p in lstm_optimizer if not any(nd in n for nd in no_decay)],
      'lr': 1e-4, 'weight_decay': 0.01},
    {'params': [p for n, p in lstm_optimizer if any(nd in n for nd in no_decay)],
      'lr': 1e-4, 'weight_decay': 0.0},
    {'params': [p for n, p in classifier_optimizer if not any(nd in n for nd in no_decay)],
      'lr': 1e-4, 'weight_decay': 0.01},
    {'params': [p for n, p in classifier_optimizer if any(nd in n for nd in no_decay)],
      'lr': 1e-4, 'weight_decay': 0.0},
    {'params': model.crf.parameters(), 'lr': 5e-4}]

epoch_num = 2
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4, correct_bias=False)
train_size = len(training_set)
train_steps_per_epoch = train_size // 32
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps=train_steps_per_epoch,
                                            num_training_steps=epoch_num * train_steps_per_epoch)
print("--------Start Training!--------")
train_model(train_loader_ext, trial_loader, model, optimizer, scheduler, epoch_num)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAvgNER: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertAvgNER from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertAvgNER from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertAvgNER were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['lstm.weight_ih_l0', 'lstm.we

cuda
--------Start Training!--------


100%|██████████| 311/311 [03:05<00:00,  1.68it/s]


Epoch: 1, train loss: 9.082063450786462


100%|██████████| 22/22 [00:04<00:00,  4.66it/s]


Epoch: 1, f1: 65.56598013581142


100%|██████████| 311/311 [03:04<00:00,  1.68it/s]


Epoch: 2, train loss: 5.881956423557935


100%|██████████| 22/22 [00:04<00:00,  4.70it/s]

Epoch: 2, f1: 66.47575346149614
Training Finished!


In [18]:
test_model(test_loader, model, optimizer, scheduler)

100%|██████████| 63/63 [00:13<00:00,  4.68it/s]

test f1: 67.48991289518419


In [ ]:
# torch.save(model.state_dict(), "/content/drive/My Drive/5018_ToxicSpans/cache_model/bert_lstm_avg_pseudo.pth")

In [ ]:
#@title ensemble { form-width: "1px" }
def test_emsemble_model(test_loader, model1, model2, model3):
  model1.eval()
  model2.eval()
  pred_tags_1 = []
  pred_tags_2 = []
  pred_tags_3 = []
  pred_labels_1 = []
  pred_labels_2 = []
  pred_labels_3 = []
  pred_labels = []
  true_labels = []
  for idx, batch_samples in enumerate(tqdm(test_loader)):
    batch_data, batch_masks, batch_labels, sentence, origional_sentences, origional_labels, length, offset = batch_samples
    # shift tensors to GPU if available
    batch_data = batch_data.to(device)
    batch_masks = batch_masks.to(device)
    batch_labels = batch_labels.to(device)

    batch_output_1 = model1(batch_data,
                        token_type_ids=None, 
                        attention_mask=batch_masks,
                        batch_len = length)
    batch_output_1= batch_output_1[0].detach().cpu().numpy()

    batch_output_2 = model2(batch_data,
                        token_type_ids=None, 
                        attention_mask=batch_masks,
                        batch_len = length)
    batch_output_2= batch_output_2[0].detach().cpu().numpy()

    batch_output_3 = model3(batch_data,
                        token_type_ids=None, 
                        attention_mask=batch_masks,
                        batch_len = length)
    batch_output_3= batch_output_3[0].detach().cpu().numpy()

    batch_labels = batch_labels.to('cpu').numpy()

    # hard ensemble
    pred_tags_1 = [[id2label.get(idx.item()) for idx in indices] for indices in np.argmax(batch_output_1, axis=2)]
    pred_tags_2 = [[id2label.get(idx.item()) for idx in indices] for indices in np.argmax(batch_output_2, axis=2)]
    pred_tags_3 = [[id2label.get(idx.item()) for idx in indices] for indices in np.argmax(batch_output_3, axis=2)]

    for i in range(len(length)):
      data_tobe_convert = {}
      data_tobe_convert["length"] = length[i]
      data_tobe_convert["offset"] = offset[i]
      data_tobe_convert["sentence"] = origional_sentences[i]
      data_tobe_convert["words"] = sentence[i]

      data_tobe_convert["tags"] = ["O"] + pred_tags_1[i]
      pred_labels_1.append(get_char_level_label(data_tobe_convert))
      data_tobe_convert["tags"] = ["O"] + pred_tags_2[i]
      pred_labels_2.append(get_char_level_label(data_tobe_convert))
      data_tobe_convert["tags"] = ["O"] + pred_tags_3[i]
      pred_labels_3.append(get_char_level_label(data_tobe_convert))

      true_labels.append(origional_labels[i])

  for i in range(len(pred_labels_1)):
    labels = []

    tmp = []
    # tmp.extend(pred_labels_1[i])
    tmp.extend(pred_labels_2[i])
    tmp.extend(pred_labels_3[i])
    
    set_tag = set(tmp)

    # vote (>50%)
    for tag in set_tag:
      if tmp.count(tag) > 1:
        labels.append(tag)

    # # union
    # labels = list(set_tag)

    # # intersection
    # for tag in set_tag:
    #   if tmp.count(tag) > 2:
    #     labels.append(tag)



    pred_labels.append(labels)

  scores = [] 
  for i in range(len(pred_labels)):
    scores.append(f1(pred_labels[i],true_labels[i]))

  scores = np.mean(scores)

  print("test f1: {}".format(scores * 100))

model_bert = BertNER.from_pretrained("bert-base-uncased")
model_bert.load_state_dict(torch.load("/content/drive/My Drive/5018_ToxicSpans/cache_model/bert.pth", map_location=torch.device('cpu')))
model_bert.to(device)

model_bert_lstm = BertLstmNER.from_pretrained("bert-base-uncased")
model_bert_lstm.load_state_dict(torch.load("/content/drive/My Drive/5018_ToxicSpans/cache_model/bert_lstm.pth", map_location=torch.device('cpu')))
model_bert_lstm.to(device)

model_bert_avg = BertAvgNER.from_pretrained("bert-base-uncased")
model_bert_avg.load_state_dict(torch.load("/content/drive/My Drive/5018_ToxicSpans/cache_model/bert_lstm_avg.pth", map_location=torch.device('cpu')))
model_bert_avg.to(device)

test_emsemble_model(test_loader, model_bert, model_bert_lstm, model_bert_avg)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertNER: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertNER from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertNER from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertNER were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', '

test f1: 68.18630110565009


In [ ]:
#@title output_label { form-width: "1px" }
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# model_bert = BertNER.from_pretrained("bert-base-uncased")
# model_bert.load_state_dict(torch.load("/content/drive/My Drive/5018_ToxicSpans/cache_model/bert.pth", map_location=torch.device('cpu')))

# model_bert_lstm = BertLstmNER.from_pretrained("bert-base-uncased")
# model_bert_lstm.load_state_dict(torch.load("/content/drive/My Drive/5018_ToxicSpans/cache_model/bert_lstm.pth", map_location=torch.device('cpu')))

# model_bert_avg = BertAvgNER.from_pretrained("bert-base-uncased")
# model_bert_avg.load_state_dict(torch.load("/content/drive/My Drive/5018_ToxicSpans/cache_model/bert_lstm_avg.pth", map_location=torch.device('cpu')))

model_bert_pseudo = BertAvgNER.from_pretrained("bert-base-uncased")
model_bert_pseudo.load_state_dict(torch.load("/content/drive/My Drive/5018_ToxicSpans/cache_model/bert_lstm_avg_pseudo.pth", map_location=torch.device('cpu')))

model_out = model_bert_pseudo
model_out.to(device)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, collate_fn=test_set.collate_fn)

pred_labels = []

for idx, batch_samples in enumerate(tqdm(test_loader)):
  batch_data, batch_masks, batch_labels, sentence, origional_sentences, origional_labels, length, offset = batch_samples
  batch_data = batch_data.to(device)
  batch_masks = batch_masks.to(device)
  batch_labels = batch_labels.to(device)

  batch_output = model_out(batch_data,
               token_type_ids=None, 
               attention_mask=batch_masks,
               batch_len = length)
  batch_output = batch_output[0].detach().cpu().numpy()
  pred_tags = [[id2label.get(idx.item()) for idx in indices] for indices in np.argmax(batch_output, axis=2)]

  for i in range(len(length)):
    data_tobe_convert = {}
    data_tobe_convert["length"] = length[i]
    data_tobe_convert["offset"] = offset[i]
    data_tobe_convert["sentence"] = origional_sentences[i]
    data_tobe_convert["words"] = sentence[i]
    data_tobe_convert["tags"] = ["O"] + pred_tags[i]
    pred_labels.append(get_char_level_label(data_tobe_convert))

# with open("/content/drive/My Drive/5018_ToxicSpans/label_files/bert_pseudo.csv", "w") as f:
#   for label_list in pred_labels:
#     for tag in label_list:
#       f.write(str(tag) + ",")
#     f.write("\n")
# f.close()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAvgNER: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertAvgNER from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertAvgNER from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertAvgNER were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['lstm.weight_ih_l0', 'lstm.we

# 66.22 bert + epoch 3
# 67.46 bert + lstm + epoch 3
# 67.81 bert + lstm + 4_8_12 avg + epoch 2
# 67.62 bert + lstm + 4_8_12 avg + pseudo_label + epoch 2
# 68.14 vote ensemble(model2 & model3)
# 68.18 intersection ensemble(model1 & model2 & model3)
